# Markowitz Efficient Frontier

## 1. Imports

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from itertools import combinations
from fetchData import fetch_raw_data_yf, getNasdaqStocks, get_matrices

## 2. Fetch Data

### Get all Nasdaq Stocks

In [2]:
assets= [
    "AAPL",  # Apple Inc.
    "MSFT",  # Microsoft Corporation
    "AMZN",  # Amazon.com Inc.
    "GOOGL", # Alphabet Inc. (Google) Class A
    "GOOG",  # Alphabet Inc. (Google) Class C
    "META",    # Meta Platforms Inc (formerly Facebook)
    "TSLA",  # Tesla Inc
    "BRK.B", # Berkshire Hathaway Inc. Class B
    "JPM",   # JPMorgan Chase & Co.
    "V",     # Visa Inc.
    "JNJ",   # Johnson & Johnson
    "WMT",   # Walmart Inc.
    "PG",    # Procter & Gamble Co.
    "UNH",   # UnitedHealth Group Inc.
    "MA",    # Mastercard Inc.
    "NVDA",  # NVIDIA Corporation
    "HD",    # Home Depot Inc.
    "BAC",   # Bank of America Corp
    "DIS",   # Walt Disney Co
    "PYPL",  # PayPal Holdings
    "VZ",    # Verizon Communications Inc.
    "ADBE",  # Adobe Inc.
    "CMCSA", # Comcast Corporation
    "NFLX",  # Netflix Inc.
    "KO",    # Coca-Cola Co
    "NKE",   # NIKE Inc.
    "PFE",   # Pfizer Inc.
    "MRK",   # Merck & Co., Inc.
    "PEP",   # PepsiCo, Inc.
    "T",     # AT&T Inc.
    "ABT",   # Abbott Laboratories
    "CRM",   # Salesforce.com Inc.
    "ORCL",  # Oracle Corporation
    "ABBV",  # AbbVie Inc.
    "CSCO",  # Cisco Systems, Inc.
    "INTC",  # Intel Corporation
    "TMO",   # Thermo Fisher Scientific Inc.
    "XOM",   # Exxon Mobil Corporation
    "ACN",   # Accenture plc
    "LLY",   # Eli Lilly and Company
    "COST",  # Costco Wholesale Corporation
    "MCD",   # McDonald's Corp
    "DHR",   # Danaher Corporation
    "MDT",   # Medtronic plc
    "NEE",   # NextEra Energy, Inc.
    "BMY",   # Bristol-Myers Squibb Company
    "QCOM",  # Qualcomm Inc
    "CVX",   # Chevron Corporation
    "WFC",   # Wells Fargo & Co
    "LMT"    # Lockheed Martin Corporation
]
assets

['AAPL',
 'MSFT',
 'AMZN',
 'GOOGL',
 'GOOG',
 'META',
 'TSLA',
 'BRK.B',
 'JPM',
 'V',
 'JNJ',
 'WMT',
 'PG',
 'UNH',
 'MA',
 'NVDA',
 'HD',
 'BAC',
 'DIS',
 'PYPL',
 'VZ',
 'ADBE',
 'CMCSA',
 'NFLX',
 'KO',
 'NKE',
 'PFE',
 'MRK',
 'PEP',
 'T',
 'ABT',
 'CRM',
 'ORCL',
 'ABBV',
 'CSCO',
 'INTC',
 'TMO',
 'XOM',
 'ACN',
 'LLY',
 'COST',
 'MCD',
 'DHR',
 'MDT',
 'NEE',
 'BMY',
 'QCOM',
 'CVX',
 'WFC',
 'LMT']

In [3]:
raw_data, asset_errors, max_combination= fetch_raw_data_yf(assets)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['BRK.B']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[***

Omitted assets( 1 ):  ['BRK.B']
Time to fetch data: 7.98 seconds


## 3. Mean and Covariance

In [4]:
Mean = get_matrices(raw_data, max_combination ,None)[0][2]
Mean

AAPL_adj_close_AAPL      0.137124
MSFT_adj_close_MSFT      0.288281
AMZN_adj_close_AMZN      0.395886
GOOGL_adj_close_GOOGL    0.263980
GOOG_adj_close_GOOG      0.278403
META_adj_close_META      0.281247
TSLA_adj_close_TSLA      0.042988
JPM_adj_close_JPM        0.209253
V_adj_close_V            0.213946
JNJ_adj_close_JNJ        0.168985
WMT_adj_close_WMT        0.151635
PG_adj_close_PG          0.087506
UNH_adj_close_UNH        0.253306
MA_adj_close_MA          0.196507
NVDA_adj_close_NVDA      0.915918
HD_adj_close_HD          0.234935
BAC_adj_close_BAC        0.237804
DIS_adj_close_DIS       -0.014722
PYPL_adj_close_PYPL      0.279233
VZ_adj_close_VZ          0.093957
ADBE_adj_close_ADBE      0.312151
CMCSA_adj_close_CMCSA    0.117291
NFLX_adj_close_NFLX      0.284074
KO_adj_close_KO          0.093521
NKE_adj_close_NKE        0.064274
PFE_adj_close_PFE        0.071060
MRK_adj_close_MRK        0.022740
PEP_adj_close_PEP        0.124627
T_adj_close_T            0.102237
ABT_adj_close_

In [5]:
Cov = get_matrices(raw_data, max_combination ,None)[0][1]
Cov

,AAPL_adj_close_AAPL,MSFT_adj_close_MSFT,AMZN_adj_close_AMZN,GOOGL_adj_close_GOOGL,GOOG_adj_close_GOOG,META_adj_close_META,TSLA_adj_close_TSLA,JPM_adj_close_JPM,V_adj_close_V,JNJ_adj_close_JNJ,...,COST_adj_close_COST,MCD_adj_close_MCD,DHR_adj_close_DHR,MDT_adj_close_MDT,NEE_adj_close_NEE,BMY_adj_close_BMY,QCOM_adj_close_QCOM,CVX_adj_close_CVX,WFC_adj_close_WFC,LMT_adj_close_LMT
AAPL_adj_close_AAPL,0.052324,0.025743,0.027016,0.022963,0.023215,0.026051,0.025790,0.019615,0.020067,0.009197,...,0.012711,0.011101,0.011572,0.012685,0.005602,0.013245,0.022603,0.015737,0.015366,0.009363
MSFT_adj_close_MSFT,0.025743,0.045053,0.032398,0.029000,0.030069,0.028604,0.024966,0.021012,0.025443,0.011835,...,0.011578,0.014765,0.015122,0.014979,0.008420,0.013891,0.021849,0.016491,0.018381,0.010453
AMZN_adj_close_AMZN,0.027016,0.032398,0.073336,0.036383,0.036730,0.043234,0.036696,0.016733,0.023472,0.010964,...,0.010285,0.015077,0.014923,0.015272,0.008734,0.014387,0.016282,0.012016,0.015596,0.007266
GOOGL_adj_close_GOOGL,0.022963,0.029000,0.036383,0.049358,0.049616,0.035587,0.026815,0.015760,0.021279,0.010312,...,0.011903,0.012525,0.011797,0.014617,0.007475,0.012362,0.015040,0.011226,0.013705,0.009055
GOOG_adj_close_GOOG,0.023215,0.030069,0.036730,0.049616,0.050982,0.035671,0.027068,0.016221,0.021241,0.010413,...,0.011679,0.012699,0.012324,0.014974,0.007267,0.012486,0.015752,0.011577,0.014292,0.009356
META_adj_close_META,0.026051,0.028604,0.043234,0.035587,0.035671,0.059810,0.034733,0.016841,0.024047,0.010238,...,0.010187,0.014042,0.014184,0.014914,0.007739,0.013903,0.008977,0.014194,0.014135,0.009363
TSLA_adj_close_TSLA,0.025790,0.024966,0.036696,0.026815,0.027068,0.034733,0.150437,0.018841,0.023035,0.009197,...,0.010143,0.011523,0.013244,0.020423,0.008848,0.017139,0.017595,0.017477,0.016637,0.008212
JPM_adj_close_JPM,0.019615,0.021012,0.016733,0.015760,0.016221,0.016841,0.018841,0.046802,0.021054,0.011455,...,0.012386,0.012336,0.010197,0.013011,0.000519,0.015851,0.021644,0.024488,0.038384,0.012045
V_adj_close_V,0.020067,0.025443,0.023472,0.021279,0.021241,0.024047,0.023035,0.021054,0.037404,0.010894,...,0.010038,0.012471,0.016066,0.017282,0.008098,0.014525,0.019083,0.014537,0.019253,0.010109
JNJ_adj_close_JNJ,0.009197,0.011835,0.010964,0.010312,0.010413,0.010238,0.009197,0.011455,0.010894,0.018506,...,0.006133,0.008582,0.011796,0.012209,0.006614,0.014186,0.010992,0.010960,0.010271,0.007545
